In [431]:
str1="Education 2017 - 2020 Aryan International College Ajmer Bachelors in Business Administration 2017 HKH Public School Ajmer Higher Secondary 2015 St. Stephens Sr. Sec. School Ajmer Certified Social Media Marketer Hubspot Senior Secondary Google Ads Certification Google"
strt= str1.replace("'","").replace("   ","").replace(", "," ").encode('ascii', errors='ignore').decode('ascii')
print(str1)
# print(strt.find("C.S))

Education 2017 - 2020 Aryan International College Ajmer Bachelors in Business Administration 2017 HKH Public School Ajmer Higher Secondary 2015 St. Stephens Sr. Sec. School Ajmer Certified Social Media Marketer Hubspot Senior Secondary Google Ads Certification Google


In [427]:
print(str1.find("Jaipur"))

60


In [433]:
print(str1[22:48])

Aryan International Colleg


In [1]:
import pandas as pd
import spacy
from collections import defaultdict
from spacy.training import Example
import ast
import random
from spacy.util import minibatch

# Load the pre-trained model
nlp = spacy.load("en_core_web_trf")

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

# Define the new entity labels
labels = ["ORG"]
for label in labels:
    ner.add_label(label)

# Load your data
df = pd.read_csv("example.csv")
merged_data = defaultdict(list)
training_data = []
# Prepare training data
for _, row in df.dropna(subset=["string", "institution", "indexes"]).iterrows():
    text = row['string']
    indexes = eval(row['indexes'])  # Convert string of list to actual list
    merged_data[text].append((indexes[0], indexes[1]+1, "ORG"))

for text, entity_list in merged_data.items():
    training_data.append((text, {"entities": entity_list}))


# print(training_data)
training_examples = []
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    training_examples.append(example)

batch_size = 4
n_iter = 20

with nlp.select_pipes(enable='ner'):
    optimizer = nlp.begin_training()
    for epoch in range(n_iter):
        random.shuffle(training_examples)
        losses = {}
        # Create minibatches and update the model
        for batch in minibatch(training_examples, size=batch_size):
            for example in batch:
                # Update the model with each example
                nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)
        print(f"Epoch {epoch + 1}/{n_iter}, Losses: {losses}")

nlp.to_disk("model2")

OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [31]:
import spacy
nlp = spacy.load("model1")

# Test the model
doc = nlp("Education 2017 - 2020 Aryan International College Ajmer Bachelors in Business Administration 2017 HKH Public School Ajmer Higher Secondary 2015 St. Stephens Sr. Sec. School Ajmer Certified Social Media Marketer Hubspot Senior Secondary Google Ads Certification Google")
for ent in doc.ents:
    print(ent.text, ent.label_)

Aryan International College ORG
HKH Public School ORG
St. Stephens Sr. Sec. ORG


In [423]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [29]:
print(training_data)

[('EDUCATION GRADUATION Bachelor Of Computer Application (BCA) Jaipur National University (JNU)   2020 - 2022 12TH Science Maths Rajasthan Board of Secondary Education   2016 - 2017', {'entities': [(60, 92, 'ORG')]}), ('Education 2017 - 2020 Aryan International College Ajmer Bachelors in Business Administration 2017 HKH Public School Ajmer Higher Secondary 2015 St. Stephens Sr. Sec. School Ajmer Certified Social Media Marketer Hubspot Senior Secondary Google Ads Certification Google', {'entities': [(22, 49, 'ORG'), (98, 115, 'ORG'), (144, 172, 'ORG')]}), ('ACADEMIC DETAILS COURSE SPECIALISATION INSTITUTE/ COLLEGE M. Tech. Instrumentation & Control Gautam Buddha University Greater Noida B. Tech Electrical Engineering Gautam Buddha University Greater Noida XII(C.B.S.E.) Science Bal Bhavan Public School Delhi X(C.B.S.E.) -- Bal Bhavan Public School Delhi %/ CGPA 6.22 6.22 68.87% 7.6 YEAR OF COMPLETION 2020 2019 2015 2013', {'entities': [(93, 117, 'ORG'), (224, 248, 'ORG')]}), ('Educationa